In [1]:
import time
import yaml
from tqdm.notebook import tqdm
from collections import defaultdict

import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

import psycopg
from psycopg import sql
from pathlib import Path
from tqdm.notebook import tqdm

필요한 계정 항목

|sj_div|sj_nm|account_nm
|---|---|---
|BS|재무상태표|유동자산
|BS|재무상태표|현금및현금성자산
|BS|재무상태표|매출채권
|BS|재무상태표|선급비용
|BS|재무상태표|재고자산
|BS|재무상태표|비유동자산
|BS|재무상태표|유형자산
|BS|재무상태표|무형자산
|BS|재무상태표|자산총계
|BS|재무상태표|유동부채
|BS|재무상태표|매입채무
|BS|재무상태표|단기차입금
|BS|재무상태표|선수금
|BS|재무상태표|비유동부채
|BS|재무상태표|사채
|BS|재무상태표|장기차입금
|BS|재무상태표|부채총계
|BS|재무상태표|자본총계
|BS|재무상태표|부채와자본총계
|IS|손익계산서|수익(매출액)
|IS|손익계산서|매출원가
|IS|손익계산서|매출총이익
|IS|손익계산서|판매비와관리비
|IS|손익계산서|영업이익
|IS|손익계산서|금융수익
|IS|손익계산서|금융비용
|IS|손익계산서|법인세비용차감전순이익(손실)
|IS|손익계산서|법인세비용
|IS|손익계산서|계속영업이익(손실)
|IS|손익계산서|당기순이익(손실)
|CF|현금흐름표|영업활동 현금흐름
|CF|현금흐름표|영업에서 창출된 현금흐름
|CF|현금흐름표|투자활동 현금흐름
|CF|현금흐름표|재무활동 현금흐름
|CF|현금흐름표|외화환산으로 인한 현금의 변동
|CF|현금흐름표|현금및현금성자산의 순증감
|CF|현금흐름표|기초의 현금및현금성자산
|CF|현금흐름표|기말의 현금및현금성자산

In [2]:
# Load settings
src_path = Path('.').absolute().parent
with (src_path / 'setting_files' / 'settings.yml').open('r') as file:
    settings = yaml.full_load(file)

dart_settings = settings['opendart']
dart = OpenDartReader(dart_settings['apikey'])

|재무제표구분|재무제표명칭|개별/연결|표시방법|세전세후
|---|---|---|---|---|
|BS1|재무상태표|연결|유동/비유동법|
|BS2|재무상태표|개별|유동/비유동법|
|BS3|재무상태표|연결|유동성배열법|
|BS4|재무상태표|개별|유동성배열법|
|IS1|별개의 손익계산서|연결|기능별분류|
|IS2|별개의 손익계산서|개별|기능별분류|
|IS3|별개의 손익계산서|연결|성격별분류|
|IS4|별개의 손익계산서|개별|성격별분류|
|CIS1|포괄손익계산서|연결|세후|
|CIS2|포괄손익계산서|개별|세후|
|CIS3|포괄손익계산서|연결|세전|
|CIS4|포괄손익계산서|개별|세전|
|DCIS1|단일 포괄손익계산서|연결|기능별분류|세후포괄손익
|DCIS2|단일 포괄손익계산서|개별|기능별분류|세후포괄손익
|DCIS3|단일 포괄손익계산서|연결|기능별분류|세전
|DCIS4|단일 포괄손익계산서|개별|기능별분류|세전
|DCIS5|단일 포괄손익계산서|연결|성격별분류|세후포괄손익
|DCIS6|단일 포괄손익계산서|개별|성격별분류|세후포괄손익
|DCIS7|단일 포괄손익계산서|연결|성격별분류|세전
|DCIS8|단일 포괄손익계산서|개별|성격별분류|세전
|CF1|현금흐름표|연결|직접법|
|CF2|현금흐름표|개별|직접법|
|CF3|현금흐름표|연결|간접법|
|CF4|현금흐름표|개별|간접법|
|SCE1|자본변동표|연결||
|SCE2|자본변동표|개별||

In [33]:
xbrl_bs = dart.xbrl_taxonomy('BS1')
xbrl_is = dart.xbrl_taxonomy('IS1')
xbrl_cis = dart.xbrl_taxonomy('CIS1')
xbrl_cf = dart.xbrl_taxonomy('CF1')

In [18]:
acc = pd.read_csv("temp.csv")

In [17]:
xbrl_bs.head()

,sj_div,bsns_de,account_id,account_nm,label_kor,label_eng,ifrs_ref,data_tp
0,BS1,20180701,ifrs_StatementOfFinancialPositionAbstract,StatementOfFinancialPositionAbstract,재무상태표 [abstract],Statement of financial position [abstract],,NaN
1,BS1,20180701,ifrs_AssetsAbstract,AssetsAbstract,자산 [abstract],Assets [abstract],,NaN
2,BS1,20180701,ifrs_CurrentAssets,CurrentAssets,유동자산,Current assets,K-IFRS 1001 문단 60,X
3,BS1,20180701,ifrs_CashAndCashEquivalents,CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,"K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45",X
4,BS1,20180701,ifrs_Cash,Cash,현금,Cash,"K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45",X


In [30]:
for a in acc.loc[acc["0"] == "BS", "1"].values:
    print(a)
    for i, (kor, eng) in xbrl_bs.loc[xbrl_bs['label_kor'].str.contains(a), ['label_kor', 'label_eng']].iterrows():
        print(f'  {kor} - {eng}')
    print()

유동자산
  유동자산 - Current assets
  기타유동자산 - Other current assets
  매각예정 또는 소유주에 대한 분배예정으로 분류된 비유동자산이나 처분자산집단 - Non-current assets or disposal groups classified as held for sale or as held for distribution to owners
  비유동자산 - Non-current assets
  기타비유동자산 - Other non-current assets

현금및현금성자산
  현금및현금성자산 - Cash and cash equivalents

매출채권
  매출채권 및 기타유동채권 - Trade and other current receivables
  매출채권 - Short-term trade Receivable
  장기매출채권 및 기타비유동채권 - Long-term trade and other non-current receivables, gross
  장기매출채권 - Long-term trade receivables, gross

선급비용
  단기선급비용 - Short-term prepaid expenses
  장기선급비용 - Long-term prepaid expenses

재고자산
  재고자산 - Inventories

비유동자산
  매각예정 또는 소유주에 대한 분배예정으로 분류된 비유동자산이나 처분자산집단 - Non-current assets or disposal groups classified as held for sale or as held for distribution to owners
  비유동자산 - Non-current assets
  기타비유동자산 - Other non-current assets

유형자산
  유형자산 - Property, plant and equipment
  기타유형자산 - Other property, plant and equipment, gross

무형자산
  영업권 이외의 무형자산 

In [34]:
for a in acc.loc[acc["0"] == "IS", "1"].values:
    print(a)
    for i, (kor, eng) in xbrl_is.loc[xbrl_is['label_kor'].str.contains(a), ['label_kor', 'label_eng']].iterrows():
        print(f'  {kor} - {eng}')
    print()

수익(매출액)

매출원가
  매출원가 - Cost of sales
  재화의 판매로 인한 수익(매출액)에 대한 매출원가 - Cost of sales from sale of goods
  제품매출원가 - Cost of sales from sale of goods product
  상품매출원가 - Cost of sales from sale of goods merchandises
  원재료매출원가 - Cost of sales from sale of goods raw material
  어획물매출원가 - Cost of sales from sale of goods fishing
  타이어매출원가 - Cost of sales from sale of goods tire
  용역의 제공으로 인한 수익(매출액)에 대한 매출원가 - Cost of sales from rendering of services
  용역매출원가 - Cost of sales from services
  게임매출원가 - Cost of sales from rendering of services games
  교육사업매출원가 - Cost of sales from rendering of services education business
  금융업매출원가 - Cost of sales from rendering of services finance business
  가스충전매출원가 - Cost of sales from sale of goods gas recharge
  방송프로그램 매출원가 - Cost of sales from rendering of services broadcast program
  물류매출원가/유통매출원가 - Cost of sales from rendering of services distribution
  임대매출원가/임대수익원가 - Cost of sales from rendering of services rental
  임가공매출원가 - Cost of sales from rendering o

/tmp/ipykernel_712/1890442730.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  for i, (kor, eng) in xbrl_is.loc[xbrl_is['label_kor'].str.contains(a), ['label_kor', 'label_eng']].iterrows():


In [47]:
xbrl_cf.loc[:, ['label_kor', 'label_eng']]

,label_kor,label_eng
0,현금흐름표 [abstract],Statement of cash flows [abstract]
1,영업활동현금흐름,Cash flows from (used in) operating activities
2,영업 현금유입,Classes of cash receipts from operating activi...
3,재화판매와 용역제공,Receipts from sales of goods and rendering of ...
4,재화판매와 용역제공의 취소,Cancel receipts from sales of goods and render...
...,...,...
221,환율변동효과 반영전 현금및현금성자산의 순증가(감소),Net increase (decrease) in cash and cash equiv...
222,현금및현금성자산에 대한 환율변동효과,Effect of exchange rate changes on cash and ca...
223,현금및현금성자산의순증가(감소),Net increase (decrease) in cash and cash equiv...
224,기초현금및현금성자산,Cash and cash equivalents at beginning of period
